In [ ]:
import os
import sys
import numpy as np 
from sklearn.cluster import KMeans
import scipy
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import pickle
import mat73
from collections import defaultdict
from pathlib import Path 
import soundfile as sf 

sys.path.append("../")
from DecayFitNet.python.toolbox.DecayFitNetToolbox import DecayFitNetToolbox
from DecayFitNet.python.toolbox.BayesianDecayAnalysis import BayesianDecayAnalysis
from DecayFitNet.python.toolbox.core import  decay_kernel, schroeder_to_envelope, PreprocessRIR, decay_model, discard_last_n_percent, FilterByOctaves
import yaml


from fade_in_reverb.data import load_simulation_dataset, load_measurement_dataset
from fade_in_reverb.analysis import load_common_decay_times, load_envelope_fit_result
config = yaml.safe_load(open("../fade_in_reverb/config.yaml"))

text_width = 7.16
column_width = 3.15

font = {'family' : 'Times New Roman',
        'size'   : 9}
params = {'text.usetex': False, 'mathtext.fontset': 'cm'}
plt.rcParams.update(params)
mpl.rc('font', **font)
plt.rcParams['text.latex.preamble'] = r"\usepackage{siunitx} \sisetup{detect-all} \usepackage{helvet} \usepackage{sansmath} \sansmath"   
# fig,axes  = plt.subplots(2, 7, figsize=(7.16, 3.2), gridspec_kw={'width_ratios': [3,0.1,3,3,3,3,0.2]})


In [ ]:
sys.path

In [ ]:
rirs, trajectory  = load_measurement_dataset()
print (rirs.shape, trajectory.shape)

In [ ]:
# first_half_rirs = rirs[:160, :] 
# second_half_rirs = rirs[160:, :] 
# print (second_half_rirs.shape)

In [ ]:
# for i in range(len(first_half_rirs)):
#     plt.plot(first_half_rirs[i])
#     plt.title(i)
#     plt.show()

In [ ]:
# All data fit 
combined_common_decay_times = load_common_decay_times("../data/hallway-lectureHall_combined_common_decay_times.npy", rirs, n_slopes=4)
print (combined_common_decay_times)

# # First half data fit 
# firstHalf_common_decay_times = load_common_decay_times("../data/hallway-lectureHall_firstHalf_common_decay_times.npy", first_half_rirs)
# print (firstHalf_common_decay_times)

# # Second half data fit 
# secondHalf_common_decay_times = load_common_decay_times("../data/hallway-lectureHall_secondHalf_common_decay_times.npy", second_half_rirs)
# print (secondHalf_common_decay_times)

In [ ]:
# # for i in range(len(second_half_rirs)):
# plt.plot(first_half_rirs[0])
# plt.show()

In [ ]:
# All data fit 
combined_pos_fit_result, combined_neg_fit_result, all_original_env = load_envelope_fit_result("../data/hallway-lectureHall_combined_model_fit_result.pkl", rirs[:, :40000], combined_common_decay_times, multichannel=False, plot=False)
print (combined_pos_fit_result.shape, combined_neg_fit_result.shape)

# # First half data fit 
# firstHalf_pos_fit_result, firstHalf_neg_fit_result, firstHalf_all_original_env = load_envelope_fit_result("../data/hallway-lectureHall_firstHalf_model_fit_result.pkl", first_half_rirs[:, :40000], firstHalf_common_decay_times, plot=False)
# print (firstHalf_pos_fit_result.shape, firstHalf_neg_fit_result.shape)

# # Second half data fit 
# secondHalf_pos_fit_result, secondHalf_neg_fit_result, secondHalf_all_original_env = load_envelope_fit_result("../data/hallway-lectureHall_secondHalf_model_fit_result.pkl", second_half_rirs[:, :40000], secondHalf_common_decay_times, plot=False)
# print (secondHalf_pos_fit_result.shape, secondHalf_neg_fit_result.shape)

# Plot amplitude transition

In [ ]:
def mu_law_encoding(x) : 
    mu = 255 
    val = np.log(1 + mu * np.abs(x)) / np.log(1 + mu)
    return np.sign(x) * val 

In [ ]:
class MidpointNormalize(mpl.colors.Normalize):
    def __init__(self, vmin, vmax, midpoint=0, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        normalized_min = max(0, 1 / 2 * (1 - abs((self.midpoint - self.vmin) / (self.midpoint - self.vmax))))
        normalized_max = min(1, 1 / 2 * (1 + abs((self.vmax - self.midpoint) / (self.midpoint - self.vmin))))
        normalized_mid = 0.5
        x, y = [self.vmin, self.midpoint, self.vmax], [normalized_min, normalized_mid, normalized_max]
        return np.ma.masked_array(np.interp(value, x, y))


In [ ]:
# mu_law_encoding(firstHalf_neg_fit_result[:, 4, 0]).shape

In [ ]:
def rotate_point(x, y, angle_rad):
    cos,sin = np.cos(angle_rad),np.sin(angle_rad)
    return cos*x-sin*y,sin*x+cos*y

def draw_brace(ax, span, position, text, text_pos, brace_scale=1.0, beta_scale=300., rotate=False, rotate_text=False):
    '''
        all positions and sizes are in axes units
        span: size of the curl
        position: placement of the tip of the curl
        text: label to place somewhere
        text_pos: position for the label
        beta_scale: scaling for the curl, higher makes a smaller radius
        rotate: true rotates to place the curl vertically
        rotate_text: true rotates the text vertically        
    '''


    # get the total width to help scale the figure
    ax_xmin, ax_xmax = ax.get_xlim()
    xax_span = ax_xmax - ax_xmin
    resolution = int(span/xax_span*100)*2+1 # guaranteed uneven
    beta = beta_scale/xax_span # the higher this is, the smaller the radius
    # center the shape at (0, 0)
    x = np.linspace(-span/2., span/2., resolution)
    # calculate the shape
    x_half = -x[:int(resolution/2)+1]
    y_half_brace = (1/(1.+np.exp(-beta*(x_half-x_half[0])))
                + 1/(1.+np.exp(-beta*(x_half-x_half[-1]))))
    y = np.concatenate((y_half_brace, y_half_brace[-2::-1]))
    # put the tip of the curl at (0, 0)
    max_y = np.max(y)    
    min_y = np.min(y)
    y /= (max_y-min_y)
    y *= brace_scale
    y -= max_y
    # rotate the trace before shifting
    if rotate:
        x,y = rotate_point(x, y, np.pi/2)
    # shift to the user's spot   
    x += position[0]        
    y += position[1]
    ax.autoscale(False)
    ax.plot(x, y, color='black', lw=1, clip_on=False)
    # put the text
    ax.text(text_pos[0], text_pos[1], text, ha='center', va='bottom', rotation=90 if rotate_text else 0) 

In [ ]:
# fig = plt.figure(layout='constrained', figsize=(text_width,text_width/2.5))
# subfigs = fig.subfigures(1, 2,  width_ratios=[1, 1], wspace=0.1)

cmap = plt.colormaps.get_cmap('RdYlBu')  # viridis is the default colormap for imshow

################## Combined ################## 
n_slopes = 4 
fig = plt.figure(figsize=(column_width,column_width* 0.8))
# fig = plt.figure(figsize=(text_width,text_width/2))

axsLeft = fig.subplots(n_slopes,1, sharex=True)

x = np.arange(0, 78, 1) 
y = np.arange(0, 2, 1)  # len = 7



# LS 2 
total_res = [] 
for i in range(77) : 
    curr_res = np.sum(combined_neg_fit_result[i*4 + 1, 2:5 ],0) 
    # curr_res = neg_fit_result[i*4 + 1 , 2]
    total_res.append(mu_law_encoding(curr_res))
total_res = np.array(total_res) 
print (total_res.shape)

for i in range(n_slopes) : 
    vmin = np.min(total_res[:, i])
    vmax = np.max(total_res[:, i]) 
    print (vmin, vmax)
    vmax = max(abs(vmin), vmax) 
    vmin = -vmax
    print (vmin, vmax)
    norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

    im = axsLeft[i].pcolormesh(x, y, total_res[:, i:i+1].T, cmap=cmap, vmin=vmin, vmax=vmax)
    divider = make_axes_locatable(axsLeft[i])
    cax = divider.append_axes("right", size="2%", pad=0.1)

#     axins = inset_axes(axsLeft[i],
#                     width="15%",  
#                     height="40%",
#                     loc='upper right',
#                        borderpad=-1.5
                       
#                    )
#     axins.yaxis.tick_right()


    if i == 3 : 
        cbar = plt.colorbar(im, cax=cax, label='Amplitude')
    else :
        cbar = plt.colorbar(im, cax=cax)
    
#     cb.ax.xaxis.set_ticks_position('top')
#     cb.ax.yaxis.set_label_position('left')
    cbar.ax.tick_params(which='major', labelsize='small', width=1, length=3)
    axsLeft[i].set_yticks([])
    axsLeft[i].set_title(rf"$\hat{{A}}_{{{i+1},\mathbf{{x}}}}$, $T_{i+1}$={np.mean(combined_common_decay_times[2:5, i]):.2f}s") 
axsLeft[-1].set_xticks([4, 12, 27, 35, 46, 61], labels=[]) # , labels=['Enter\nstairwell', 'Exit\nstairwell', 'Enter\nmeeting\nroom', 'Exit\nmeeting\nroom', 'Enter\nworkshop area', 'Enter\nlarge lecture hall\n'], rotation=0, ha='center')

draw_brace(axsLeft[-1], 8, (8,0.7), "Stairwell", (8,-1.3), brace_scale=0.3)
draw_brace(axsLeft[-1], 8, (31,0.7), "Meeting room", (33,-1.3), brace_scale=0.3)
draw_brace(axsLeft[-1], 15, (69,0.7), "Lecture hall", (69,-1.3), brace_scale=0.3)


plt.subplots_adjust(wspace=0, hspace=1.3)

plt.savefig("../figures/measurement_amplitude_transition_A.pdf", bbox_inches="tight") 

plt.show() 

# LS 4
fig = plt.figure(figsize=(column_width,column_width*0.8))
axsLeft = fig.subplots(n_slopes,1, sharex=True)


total_res = [] 
for i in range(77) : 
    curr_res = np.sum(combined_neg_fit_result[i*4 + 3, 2:5 ],0) 
    # curr_res = neg_fit_result[i*4 + 1 , 2]
    total_res.append(mu_law_encoding(curr_res))
total_res = np.array(total_res) 
print (total_res.shape)


for i in range(n_slopes) : 
    vmin = np.min(total_res[:, i])
    vmax = np.max(total_res[:, i]) 
    print (vmin, vmax)
    vmax = max(abs(vmin), vmax) 
    vmin = -vmax
    print (vmin, vmax)
    norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

    im = axsLeft[i].pcolormesh(x, y, total_res[:, i:i+1].T, cmap=cmap, vmin=vmin, vmax=vmax)
    divider = make_axes_locatable(axsLeft[i])
   
    cax = divider.append_axes("right", size="2%", pad=0.1)

#     axins = inset_axes(axsLeft[i],
#                     width="20%",  
#                     height="40%",
#                     loc='upper right',
#                     borderpad=1
#                    )
    if i == 3 : 
#         plt.colorbar(im, cax=axins, label='Amplitude', orientation='horizontal')
        cbar = plt.colorbar(im, cax=cax, label='Amplitude')
    else :
#         plt.colorbar(im, cax=axins, orientation='horizontal')
        cbar = plt.colorbar(im, cax=cax)
    
#     cbar.ax.set_yticklabels([-vmin, 0, vmax])
    cbar.ax.tick_params(which='major', labelsize='small', width=1, length=3)

    axsLeft[i].set_xlim([0,77])
    axsLeft[i].set_yticks([])
    axsLeft[i].set_title(rf"$\hat{{A}}_{{{i+1},\mathbf{{x}}}}$, $T_{i+1}$={np.mean(combined_common_decay_times[2:5, i]):.2f}s") 

axsLeft[-1].set_xticks([4, 12, 27, 35, 61, 77], labels=[]) # ['Enter\nstairwell', 'Exit\nstairwell', 'Enter\nmeeting\nroom', 'Exit\nmeeting\nroom', 'Enter\nworkshop area', 'Enter\nlarge lecture hall\n'], rotation=0, ha='center')

draw_brace(axsLeft[-1], 8, (8,0.7), "Stairwell", (8,-1.3), brace_scale=0.3)
draw_brace(axsLeft[-1], 8, (31,0.7), "Meeting room", (33,-1.3), brace_scale=0.3)
draw_brace(axsLeft[-1], 15, (69,0.7), "Lecture hall", (69,-1.3), brace_scale=0.3)


plt.subplots_adjust(wspace=0, hspace=1.3)

# plt.tight_layout()
plt.savefig("../figures/measurement_amplitude_transition_B.pdf", bbox_inches="tight") 

plt.show()


# DRR error 

In [ ]:
ls = 1
L = 40000
ds_start_index = 2
downSampleLength = 200 
timeAxis_ds = np.linspace(0, L / config['sample_rate'], downSampleLength- ds_start_index) 
timeAxis_fullLength = np.linspace(0, L/config['sample_rate'], L)


neg_error_list = [] 
pos_error_list = []

for i in range(77) : 
    rir_number = i * 4 + ls 
    true_rir = rirs[rir_number][:L]
    
    curr_rir_neg_fit_result = combined_neg_fit_result[rir_number]
    curr_rir_pos_fit_result = combined_pos_fit_result[rir_number]
#     curr_rir_original_env = combined_all_original_env[rir_number]


    neg_rir = np.zeros_like(true_rir)
    pos_rir = np.zeros_like(true_rir)
    
    for bIdx in range(len(config['f_bands'])) : 
        # Filter signal by octave
        filterbank = FilterByOctaves(order=6, sample_rate=config['sample_rate'], backend='scipy',
                                                center_frequencies=[config['f_bands'][bIdx]])
        
        # Perform octave filtering at the current octave 
        octave_filtered_rir = filterbank(torch.FloatTensor(true_rir))[0]
        octave_filtered_rir = octave_filtered_rir.numpy()

        octave_filtered_rir_norm_factor = np.max(np.abs(octave_filtered_rir)) 
        octave_filtered_rir = octave_filtered_rir / octave_filtered_rir_norm_factor

        
        curr_common_decay_times = combined_common_decay_times[bIdx]
        
        # Get the exponentials 
        envelopeTimes = 2 * curr_common_decay_times
        envelopes = decay_kernel(envelopeTimes, timeAxis_ds)
        envelopes_fullLength = decay_kernel(envelopeTimes, timeAxis_fullLength)

        # RMS for later gain matching 
        original_rms = np.sqrt(np.mean(octave_filtered_rir**2))

        # The noise part is just ones 
        envelopes[:, -1] = np.ones_like(envelopes[:, -1]) 
        envelopes_fullLength[:, -1] = np.ones_like(envelopes_fullLength[:, -1]) 

        # Neg envelope full length
        neg_weighted_envelopes = envelopes_fullLength[:, :-1]  * curr_rir_neg_fit_result[bIdx][:-1]
        neg_envelopes = np.sum(neg_weighted_envelopes, 1) 

        # Neg envelope full length + noise 
        neg_weighted_envelopes_with_noise = envelopes_fullLength  * curr_rir_neg_fit_result[bIdx]
        neg_envelopes_with_noise = np.sum(neg_weighted_envelopes_with_noise, 1) 

        # Neg envelope downsampled 
        # neg_weighted_envelopes_ds = envelopes[:, :-1]  * curr_rir_neg_fit_result[bIdx][:-1] 
        neg_weighted_envelopes_ds = envelopes  * curr_rir_neg_fit_result[bIdx]
        neg_envelopes_ds = np.sum(neg_weighted_envelopes_ds, 1) 

        pos_weighted_envelopes = envelopes_fullLength[:, :-1]  * curr_rir_pos_fit_result[bIdx][:-1] 
        pos_envelopes = np.sum(pos_weighted_envelopes, 1) 


        pos_weighted_envelopes_with_noise = envelopes_fullLength  * curr_rir_pos_fit_result[bIdx]
        pos_envelopes_with_noise = np.sum(pos_weighted_envelopes_with_noise, 1) 
        
        # pos_weighted_envelopes_ds = envelopes[:, :-1]  * curr_rir_pos_fit_result[bIdx][:-1] 
        pos_weighted_envelopes_ds = envelopes  * curr_rir_pos_fit_result[bIdx]
        pos_envelopes_ds = np.sum(pos_weighted_envelopes_ds, 1) 

#         print (curr_rir_pos_fit_result[bIdx])
#         print (curr_rir_neg_fit_result[bIdx])

        noise = np.random.randn(L*2)
        noise_rms = np.sqrt(np.mean(noise**2))
        octave_filtered_noise = filterbank(torch.FloatTensor(noise))
        octave_filtered_noise = octave_filtered_noise.numpy()[0]
        octave_filtered_noise  = octave_filtered_noise[L//2 : -L//2]

        neg_shaped_noise = neg_envelopes * octave_filtered_noise
        neg_shaped_noise_rms = np.sqrt(np.mean(neg_shaped_noise**2)) 
        neg_shaped_noise *= original_rms / neg_shaped_noise_rms 

        noise = np.random.randn(L*2)
        noise_rms = np.sqrt(np.mean(noise**2))
        octave_filtered_noise = filterbank(torch.FloatTensor(noise))
        octave_filtered_noise = octave_filtered_noise.numpy()[0]
        octave_filtered_noise  = octave_filtered_noise[L//2 : -L//2]

        pos_shaped_noise = pos_envelopes * octave_filtered_noise
        pos_shaped_noise_rms = np.sqrt(np.mean(pos_shaped_noise**2)) 
        pos_shaped_noise *= original_rms / pos_shaped_noise_rms 
        
        neg_shaped_noise  = neg_shaped_noise * octave_filtered_rir_norm_factor
        pos_shaped_noise = pos_shaped_noise * octave_filtered_rir_norm_factor

        neg_rir += neg_shaped_noise 
        pos_rir += pos_shaped_noise 
    

#     plt.plot(true_rir)
#     plt.show()
#     plt.plot(neg_rir)
#     plt.show()
#     plt.plot(pos_rir)
#     plt.show()
#     print ("--------------------------------------------------")

    direct_len = int(48000 * 0.05)

    true_drr = 10 * np.log10 (np.sum(true_rir[:direct_len] ** 2) / np.sum(true_rir[direct_len:] **2))
    neg_drr = 10 * np.log10 (np.sum(neg_rir[:direct_len] ** 2) / np.sum(neg_rir[direct_len:] **2))
    pos_drr = 10 * np.log10 (np.sum(pos_rir[:direct_len] ** 2) / np.sum(pos_rir[direct_len:] **2))
    
    neg_error = abs(true_drr- neg_drr)
    pos_error = abs(true_drr - pos_drr)
    
    neg_error_list.append(neg_error)
    pos_error_list.append(pos_error) 
    


In [ ]:
fig, ax = plt.subplots(figsize=(column_width,column_width* 0.7))
plt.plot(neg_error_list, label="fade-in", color='C3')
plt.plot(pos_error_list, label="pos-only", color='C10')
plt.ylim([0, 11])
plt.legend()
ax.set_xticks([4, 12, 27, 35, 61, 77], labels=[]) # ['Enter\nstairwell', 'Exit\nstairwell', 'Enter\nmeeting\nroom', 'Exit\nmeeting\nroom', 'Enter\nworkshop area', 'Enter\nlarge lecture hall\n'], rotation=0, ha='center')
plt.ylabel(rf'$C_{{50}}$ error in dB')
draw_brace(ax, 8, (8,0.5), "Stairwell", (8,-2), brace_scale=0.3)
draw_brace(ax, 8, (31,0.5), "Meeting room", (32,-2), brace_scale=0.3)
draw_brace(ax, 15, (69,0.5), "Lecture hall", (69,-2), brace_scale=0.3)
ax.fill_between(range(27,36), 11, color="grey", alpha=0.3, zorder=-100)

# plt.tight_layout()
plt.savefig("../figures/measurement_ls2_drr.pdf", bbox_inches="tight") 


In [ ]:

# ################### First half ###################
# fig = plt.figure(figsize=(column_width,column_width/1.1))
# axsLeft = fig.subplots(3,1, sharex=True)

# x = np.arange(0, 41, 1) 
# y = np.arange(0, 2, 1)  # len = 7

# firstHalf_res = [] 
# for i in range(40) : 
#     curr_res = np.sum(firstHalf_neg_fit_result[i*4 + 1, 2:5 ],0) 
#     # curr_res = neg_fit_result[i*4 + 1 , 2]
#     firstHalf_res.append(mu_law_encoding(curr_res))
# firstHalf_res = np.array(firstHalf_res) 
# print (firstHalf_res.shape)

# # A0 
# vmin = np.min(firstHalf_res[:, 0])
# vmax = np.max(firstHalf_res[:, 0]) 
# print (vmin, vmax)
# vmax = max(abs(vmin), vmax) 
# vmin = -vmax
# print (vmin, vmax)
# norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

# im = axsLeft[0].pcolormesh(x, y, firstHalf_res[:, 0:1].T, cmap=cmap, vmin=vmin, vmax=vmax)
# divider = make_axes_locatable(axsLeft[0])
# cax = divider.append_axes("right", size="2%", pad=0.1)
# plt.colorbar(im, cax=cax)
# axsLeft[0].set_yticks([])
# axsLeft[0].set_title(rf"$A_0$, $T_0$={np.mean(firstHalf_common_decay_times[2:5, 0]):.2f}s") 


# # A1 
# vmin = np.min(firstHalf_res[:, 1])
# vmax = np.max(firstHalf_res[:, 1]) 
# print (vmin, vmax)
# vmax = max(abs(vmin), vmax) 
# vmin = -vmax
# print (vmin, vmax)
# norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

# im = axsLeft[1].pcolormesh(x, y, firstHalf_res[:, 1:2].T, cmap=cmap, vmin=vmin, vmax=vmax)
# divider = make_axes_locatable(axsLeft[1])
# cax = divider.append_axes("right", size="2%", pad=0.1)
# plt.colorbar(im, cax=cax,  label=fr'Amplitude')
# axsLeft[1].set_yticks([])
# axsLeft[1].set_title(rf"$A_0$, $T_0$={np.mean(firstHalf_common_decay_times[2:5, 1]):.2f}s") 


# # A2 
# vmin = np.min(firstHalf_res[:, 2])
# vmax = np.max(firstHalf_res[:, 2]) 
# print (vmin, vmax)
# vmax = max(abs(vmin), vmax) 
# vmin = -vmax
# print (vmin, vmax)
# norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

# im = axsLeft[2].pcolormesh(x, y, firstHalf_res[:, 2:3].T, cmap=cmap, vmin=vmin, vmax=vmax)
# divider = make_axes_locatable(axsLeft[2])
# cax = divider.append_axes("right", size="2%", pad=0.1)
# plt.colorbar(im, cax=cax)
# axsLeft[2].set_yticks([])
# axsLeft[2].set_title(rf"$A_0$, $T_0$={np.mean(firstHalf_common_decay_times[2:5, 2]):.2f}s") 

# # subfigs[0].suptitle('Position 0 to 40, Loudspeaker 2', fontsize='x-large')
# # subfigs[0].colorbar(pc, shrink=0.6, ax=axsLeft, location='bottom')


# axsLeft[2].set_xticks([4, 12, 27, 35], labels=['Enter\nstairwell', 'Exit\nstairwell', 'Enter\nmeeting\nroom', 'Exit\nmeeting\nroom'], rotation=0, ha='center')
# axsLeft[2].set_xlabel("Measurement position")

# plt.tight_layout()
# plt.savefig("measurement_amplitude_transition_A.pdf", bbox_inches="tight") 
# plt.show() 


# ################### Second half ###################
# fig = plt.figure(figsize=(column_width,column_width/1.1))

# axsRight = fig.subplots(3, 1, sharex=True)

# x = np.arange(0, 38, 1) 
# y = np.arange(0, 2, 1)  # len = 7

# secondHalf_res = [] 
# for i in range(37) : 
#     curr_res = np.sum(secondHalf_neg_fit_result[i*4 + 3, 2:5],0) 
#     # curr_res = neg_fit_result[i*4 + 1 , 4]
#     secondHalf_res.append(mu_law_encoding(curr_res))
# secondHalf_res = np.array(secondHalf_res) 
# print (secondHalf_res.shape)

# # A0 
# vmin = np.min(secondHalf_res[:, 0])
# vmax = np.max(secondHalf_res[:, 0]) 
# print (vmin, vmax)
# vmax = max(abs(vmin), vmax) 
# vmin = -vmax
# print (vmin, vmax)
# norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

# im = axsRight[0].pcolormesh(x, y, secondHalf_res[:, 0:1].T, cmap=cmap, vmin=vmin, vmax=vmax)
# divider = make_axes_locatable(axsRight[0])
# cax = divider.append_axes("right", size="2%", pad=0.1)
# plt.colorbar(im, cax=cax)
# axsRight[0].set_yticks([])
# axsRight[0].set_title(rf"$A_0$, $T_0$={np.mean(secondHalf_common_decay_times[2:5, 0]):.2f}s") 


# # A1 
# vmin = np.min(secondHalf_res[:, 1])
# vmax = np.max(secondHalf_res[:, 1]) 
# print (vmin, vmax)
# vmax = max(abs(vmin), vmax) 
# vmin = -vmax
# print (vmin, vmax)
# norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

# im = axsRight[1].pcolormesh(x, y, secondHalf_res[:, 1:2].T, cmap=cmap, vmin=vmin, vmax=vmax)
# divider = make_axes_locatable(axsRight[1])
# cax = divider.append_axes("right", size="2%", pad=0.1)
# plt.colorbar(im, cax=cax, label=fr'Amplitude')
# axsRight[1].set_yticks([])
# axsRight[1].set_title(rf"$A_0$, $T_0$={np.mean(secondHalf_common_decay_times[2:5, 1]):.2f}s") 


# # A2 
# vmin = np.min(secondHalf_res[:, 2])
# vmax = np.max(secondHalf_res[:, 2]) 
# print (vmin, vmax)
# vmax = max(abs(vmin), vmax) 
# vmin = -vmax
# print (vmin, vmax)
# norm = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

# im = axsRight[2].pcolormesh(x, y, secondHalf_res[:, 2:3].T, cmap=cmap, vmin=vmin, vmax=vmax)
# divider = make_axes_locatable(axsRight[2])
# cax = divider.append_axes("right", size="2%", pad=0.1)
# plt.colorbar(im, cax=cax)
# axsRight[2].set_yticks([])
# axsRight[2].set_title(rf"$A_0$, $T_0$={np.mean(secondHalf_common_decay_times[2:5, 2]):.2f}s") 
# axsRight[2].set_xlabel("Measurement position") 

# # subfigs[1].suptitle('Position 41 to 77, Loudspeaker 4', fontsize='x-large')
# # subfigs[0].colorbar(pc, shrink=0.6, ax=axsLeft, location='bottom')


# axsRight[2].set_xticks([6, 21], labels=['Enter\nworkshop area', 'Enter\nlarge lecture hall\n'], rotation=0, ha='center')

# plt.tight_layout()

# plt.savefig("measurement_amplitude_transition_B.pdf", bbox_inches="tight") 

# Plot some samples 

In [ ]:
# fig, axes = plt.subplots(2,1, sharey=True, sharex=True, figsize=(5,7))
# axes[0].plot(omni_rirs[197, :48000])
# axes[1].plot(omni_rirs[750, :48000])
# from PIL import Image

# plt.style.use('default')

pos1 = 30 
pos2 = 22
pos3 = 10

ls = 1 

end_index = 20000 

# fig = plt.figure(figsize=(8,3))
# gs = plt.GridSpec(nrows=1, ncols=2, width_ratios=[1.5, 1],wspace=0.4)



# ax0 = fig.add_subplot(gs[0], aspect='equal')
# # img = plt.imread("/Users/kyungyunlee/Downloads/hallway-lecturehall_map_cropped.png")
# img = Image.open("/Users/kyungyunlee/Downloads/hallway-lecturehall_map_cropped.png", mode='r', formats=None) 
# imgplot = ax0.imshow(img)
# ax0.set_axis_off()
# ax2 = fig.add_subplot(gs[:, 0], aspect='equal')
# # axins = ax0.inset_axes(
# #     [0.6, 0.54, 0.45, 0.45], xticklabels=[], yticklabels=[], aspect='equal')
# plot_room_geometry(ax2)
# ax2.scatter(srcPos[0], srcPos[1], marker='x', linewidth=1, color='black')
# ax2.scatter(rcvPos1[0], rcvPos1[1], marker='*',linewidth= 1,  color='C0')
# ax2.scatter(rcvPos2[0], rcvPos2[1], marker='*', linewidth= 1,color='C1')
# ax2.scatter(rcvPos3[0], rcvPos3[1], marker='*', linewidth=1,color='C2')
# ax2.set_title('a) Room layout')
# ax2.set_xlabel("x in meters")
# ax2.set_ylabel("y in meters")
# # ax2.text( 5, 0.7, "Source")
# ax2.annotate("Source",
#             xy=(2.5, 2), xycoords='data',
#             xytext=(5, 0.5), textcoords='data',
#             arrowprops=dict(arrowstyle="->", connectionstyle="arc3"))

# ax2.text( 2, 4, "Room 1\n(Short\nT60)",  horizontalalignment='center')
# ax2.text( 7, 3.2, "Room 2\n(Long T60)", horizontalalignment='center')
# ax2.text( 8, 7, "Room 3\n(Mid\nT60)", horizontalalignment='center')
# ax2.fill_betweenx([0,8],[0,0],[4,4], alpha=0.2)
# ax2.fill_betweenx([2,5],[4,4],[10,10], alpha=0.2)
# ax2.fill_betweenx([5,13],[6,6],[10,10], alpha=0.2)


fig = plt.figure(figsize=(column_width, column_width*0.8))
ax1 = plt.gca()

xaxis = np.arange(0, end_index/48000, 1/48000) 


test_edf = np.flipud(np.cumsum(np.flipud(first_half_rirs[pos1*4 + ls][50:96050]**2)))
ax1.plot(xaxis,(10 * np.log10(test_edf))[:end_index], label=f"Meeting room (Pos {pos1})")
test_edf = np.flipud(np.cumsum(np.flipud(first_half_rirs[pos2*4 + ls][:96000]**2)))
ax1.plot(xaxis,(10 * np.log10(test_edf))[:end_index], label=f"Hallway (Pos {pos2})")
test_edf = np.flipud(np.cumsum(np.flipud(first_half_rirs[pos3*4 + ls][600:96600]**2)))
ax1.plot(xaxis,(10 * np.log10(test_edf))[:end_index], label=f"Stairwell (Pos {pos3})")

ax1.legend()




ax1.set_ylabel("Decay level in dB")
# ax0.set_xticks(list(np.arange(0, 48000,1)))
ax1.set_xlabel("Time in seconds")
# ax1.set_title("Energy decay curves")
# ax0.set_ylim([-60, 0]) 
# ax0.set_xlim([0, 1.0]) 
plt.savefig("../figures/measurement_edc_rirs_A.pdf", bbox_inches="tight") 

plt.show() 


fig,axes = plt.subplots(3, 1, figsize=(column_width, column_width * 0.7))
plt.subplots_adjust(hspace=0.2)



# ax2 = fig.add_subplot(gs[2])
# gp = gs[1].subgridspec(3, 1)

# ax2 = fig.add_subplot(gp[0])
# ax3 = fig.add_subplot(gp[1])
# ax4 = fig.add_subplot(gp[2])

# ax2 = fig.add_subplot(gs[2], sharex=ax1)
# ax3 = fig.add_subplot(gs[3], sharex=ax1)

ax2, ax3, ax4 = axes 

ax2.plot(xaxis,first_half_rirs[pos1*4 + ls, 50:end_index+50], label=f"Room 1")
room1_max = np.max(np.abs(first_half_rirs[pos1*4 + ls]))
vmax= room1_max * 1.2
ax2.set_ylim([-vmax, vmax])

ax3.plot(xaxis,first_half_rirs[pos2*4 + ls, :end_index], label=f"Room 2", color='C1')
room2_max = np.max(np.abs(first_half_rirs[pos2*4 + ls]))
vmax= room2_max * 1.2
ax3.set_ylim([-vmax, vmax])

ax4.plot(xaxis,first_half_rirs[pos3*4 + ls, 600:end_index+600], label=f"Room 3", color='C2')
room3_max = np.max(np.abs(first_half_rirs[pos3*4 + ls]))
vmax= room3_max * 1.2
ax4.set_ylim([-vmax, vmax])
# ax1.legend() # ncols=1, loc='upper center', bbox_to_anchor=(0.5, 1))
# ax1.tick_params(axis='y',  left=False, labelleft=False)
ax2.set_xticks([])
ax3.set_xticks([])

# ax1 = fig.add_subplot(gs[0, 1])
# ax1.plot(omni_rirs[197, :48000])
# ax1.set_title(f"Room 1 : x={rcvPos1[0]}m, y={rcvPos1[1]}m")
# # ax1.set_xlim([0, 48000])

# ax2 = fig.add_subplot(gs[1, 1], sharey=ax1, sharex=ax1)
# ax2.plot(omni_rirs[384, :48000])
# ax2.set_title(f"Room 2 : x={rcvPos2[0]}m, y={rcvPos2[1]}m")


# ax3 = fig.add_subplot(gs[2, 1], sharey=ax1, sharex=ax1)
# ax3.plot(omni_rirs[750, :48000])
# ax3.set_title(f"Room 3 : x={rcvPos3[0]}m, y={rcvPos3[1]}m")
# ax1.set_ylabel("Amplitude") 
# ax1.set_xlabel("Time in seconds")
# ax1.set_title("RIR in meeting room") 

ax3.set_ylabel("Signal Value") 
# ax2.set_xlabel("Time in seconds")
# ax2.set_ylim([-0.02, 0.02])
# ax2.set_title("RIRs") 

ax4.set_xlabel("Time in seconds") 
# ax3.set_xlabel("Time in seconds")
# ax3.set_ylim([-0.006, 0.006])
# ax3.set_title("RIR in the stairwell") 
# params = dict(bottom=0, left=0, right=1)
# ax1.subplots_adjust(**params)
# plt.tight_layout()
# # ax3 = fig.add_axes([0.6, 0.6, 0.2, 0.2])
# # ax3.hist(distribution)
# # plt.show()

# fig.savefig(fname='measurement_edc_rirs.pdf', format='pdf', bbox_inches="tight") 
plt.savefig("../figures/measurement_edc_rirs_B.pdf", bbox_inches="tight") 




In [ ]:
# Load position info 
    
rir_df = pickle.load(open("/Users/kyungyunlee/dev/blind-measurement/all_scenes_with_position_data.pkl", "rb"))
scene_df = rir_df[rir_df['scene'] == 'offices']

trajectory = list(zip(list(scene_df['posX']),list(scene_df['posY']), list(scene_df['posZ'])))

trajectory = np.array(trajectory)

In [ ]:
ax = plt.axes()
end_index = -1
ax.plot(trajectory[:end_index, 0],trajectory[:end_index,2], color='forestgreen', zorder=-1)
ax.scatter(trajectory[:, 0], trajectory[:, 2], color='blueviolet', zorder=1, alpha=0.8)
# ax.scatter(trajectory[40:end_index, 0], trajectory[40:end_index, 2], color='lightseagreen', zorder=1, alpha=0.8)
ax.set_aspect("equal")
ax.set_xlim([-5, 10])
ax.set_ylim([-10, 12])
plt.savefig("../figures/trajectory.png", dpi=1000, transparent=True) 